In [1]:
import sys
import os
shogun_path="/usr/local/lib/python2.7/site-packages/"
sys.path.append(shogun_path)
print(sys.path)
from modshogun import *

['', '/home/global/anaconda2/lib/python27.zip', '/home/global/anaconda2/lib/python2.7', '/home/global/anaconda2/lib/python2.7/plat-linux2', '/home/global/anaconda2/lib/python2.7/lib-tk', '/home/global/anaconda2/lib/python2.7/lib-old', '/home/global/anaconda2/lib/python2.7/lib-dynload', '/home/romain/.local/lib/python2.7/site-packages', '/home/global/anaconda2/lib/python2.7/site-packages', '/home/global/anaconda2/lib/python2.7/site-packages/Sphinx-1.5.6-py2.7.egg', '/home/global/anaconda2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '/home/global/anaconda2/lib/python2.7/site-packages/IPython/extensions', '/home/romain/.ipython', '/usr/local/lib/python2.7/site-packages/']


In [2]:
%pylab inline
%matplotlib inline
from scipy.io import loadmat, savemat
from os       import path, sep

Populating the interactive namespace from numpy and matplotlib


In [3]:
kernel = CombinedKernel()

In [4]:
mat  = loadmat('/NAS/dumbo/protocoles/CogPhenoPark/data/cogphenoparkCli.mat')
Xall = mat['dataCli']
Yall = array(mat['label'].squeeze(), dtype=double)
print Xall.shape
#Yall = Yall - 1
random.seed(0)
subset = random.permutation(len(Yall))
print(subset[1:len(Yall)/2])
print(subset[len(Yall)/2:len(Yall)])

IOError: [Errno 2] No such file or directory: '/NAS/dumbo/protocoles/CogPhenoPark/data/cogphenoparkCli.mat'

In [9]:
Xtrain = Xall[subset[1:len(Yall)/2],:]
Ytrain = Yall[subset[1:len(Yall)/2]]
labels = MulticlassLabels(Ytrain)
feats  = RealFeatures(Xtrain)

Xrem=Xall[subset[len(Yall)/2:len(Yall)],:]
Yrem=Yall[subset[len(Yall)/2:len(Yall)]]
labels_rem=MulticlassLabels(Yrem)
feats_rem=RealFeatures(Xrem)      

In [82]:
poly_kernel = PolyKernel(10, 2)
gauss_kernel_1 = GaussianKernel(2.0)
gauss_kernel_2 = GaussianKernel(3.0)

combined_kernel = CombinedKernel()
combined_kernel.append_kernel(poly_kernel)
combined_kernel.append_kernel(gauss_kernel_1)
combined_kernel.append_kernel(gauss_kernel_2)
combined_kernel.init(feats, feats)

True

In [1]:
binary_svm_solver = SVRLight()
mkl = MKLRegression(binary_svm_solver)
mkl.set_kernel(combined_kernel)
mkl.set_labels(labels)
mkl.train()

NameError: name 'SVRLight' is not defined

In [ ]:
#Nsplit = 2
#all_ks = range(2, 21)
# MKL training and output

feats_train = CombinedFeatures()
feats_test = CombinedFeatures()

#append gaussian kernel
subkernel = GaussianKernel(10,15)        
feats_train.append_feature_obj(feats)
feats_test.append_feature_obj(feats_rem)
kernel.append_kernel(subkernel)

#append PolyKernel
subkernel = PolyKernel(10,2)            
feats_train.append_feature_obj(feats)
feats_test.append_feature_obj(feats_rem)
kernel.append_kernel(subkernel)

kernel.init(feats_train, feats_train)
mkl = MKLMulticlass(1.2, kernel, labels)

mkl.set_epsilon(1e-2)
mkl.set_mkl_epsilon(0.001)
mkl.set_mkl_norm(1)

mkl.train()


In [20]:
C = 1.0
epsilon = 0.0001
gauss_kernel = GaussianKernel(feats, feats, 1)
svm = MulticlassLibSVM(C, gauss_kernel, labels)
svm.set_epsilon(epsilon)
svm.train()
#labels_predict = svm.apply_multiclass(feats_rem)

/mnt/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: [WARN] In file /home/romain/Downloads/shogun_6.0.0/src/shogun/multiclass/MulticlassOneVsOneStrategy.cpp line 34: MulticlassOneVsOneStrategy::CMulticlassOneVsOneStrategy(): register parameters!

  after removing the cwd from sys.path.


SystemError: [1;31m[ERROR][0m In file /home/romain/Downloads/shogun_6.0.0/src/shogun/kernel/Kernel.h line 211: GaussianKernel::kernel(): index out of Range: idx_a=11/3 idx_b=11/3
